# Initialize

In [1]:
!pip install nltk

You should consider upgrading via the 'c:\users\lhustvei\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
# Does the input string contain a digit?
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

hasNumbers('4444')

True

In [3]:
#
# Lastar inn teksten vi jobbar med
#

with open('./input - Chapter 3 - Text from word document with correct formating.txt', 'r') as f:

    forward_file_lines = f.readlines() 
    
print(forward_file_lines)

['3. Fire, machinery and electric installations\n', '\n', 'Section 7. Scope of application of this chapter\n', '(1) This chapter applies to vessels constructed after 2 January 1988.\n', '(2) Vessels constructed before 2 January 1988 shall be constructed and equipped in accordance with the rules applicable at the time of construction of the vessel, unless otherwise provided by the individual provisions of this chapter.\n', '(3) When a vessel is substantially altered or modified, the vessel shall comply with the requirements for a new vessel.\n', '\n', '0\tAmended by Regulation of 4 May 2015 No.. 445 (in force on 6 May 2015).\n', '\n', '\n', '\n', 'Section 8. Fire safety\n', '(1) Vessels constructed after 1 January 1992 shall have structural fire protection in accordance with chapter C14 of Nordic Boat Standard (1990), unless otherwise provided by the individual provisions of this chapter.\n', '(2) Vessels constructed before 1 January 1992 shall have structural fire protection in accorda

# Pull out info from chapter text

In [4]:
from nltk.tokenize import word_tokenize

forward_headline_and_section_list = []

forward_chapter_info = {}

for line_index_number, line_text in enumerate(forward_file_lines):

    word_tokens = word_tokenize(line_text, language="english")

    sentence_word_count = len(word_tokens)

    if sentence_word_count > 3 and word_tokens[0] == 'Section' and hasNumbers(word_tokens[1]) and word_tokens[2] == '.':
        sentence_type = 'headline-section'
        type_value = word_tokens[1] # section number
        headline_in_a_list = word_tokens[3:sentence_word_count]
        headline_as_a_string = " ".join(headline_in_a_list)
        forward_headline_and_section_list.append((str(line_index_number),sentence_type,type_value,line_text.replace("\n","")))

    # headline-chapter
    elif "chapter_title" not in forward_chapter_info and sentence_word_count > 2 and line_index_number == 0 and hasNumbers(word_tokens[0]):

        forward_chapter_info['chapter_title'] = line_text.replace("\n","")
        forward_chapter_info['chapter_number'] = word_tokens[0]

#print(forward_headline_and_section_list)
forward_chapter_info

{'chapter_title': '3. Fire, machinery and electric installations',
 'chapter_number': '3'}

In [12]:
forward_identified_sentence_type_list = []

for line_index_number, line_text in enumerate(forward_file_lines):

    word_tokens = word_tokenize(line_text, language="english")

    sentence_word_count = len(word_tokens)

    if sentence_word_count > 3 and word_tokens[0] == '(' and hasNumbers(word_tokens[1]) and word_tokens[2] == ')':
        sentence_type = 'part'
        type_value = word_tokens[1] # part number
        forward_identified_sentence_type_list.append((str(line_index_number),sentence_type,type_value))

    elif sentence_word_count > 3 and word_tokens[0] == 'Section' and hasNumbers(word_tokens[1]) and word_tokens[2] == '.':
        sentence_type = 'headline-section'
        type_value = word_tokens[1] # section number
        forward_identified_sentence_type_list.append((str(line_index_number),sentence_type,type_value))

    elif sentence_word_count > 2 and word_tokens[1] == ')':
        sentence_type = 'sub-part'
        type_value = word_tokens[0] # sub-part character
        forward_identified_sentence_type_list.append((str(line_index_number),sentence_type,type_value))

    elif sentence_word_count > 2 and line_index_number == 0 and hasNumbers(word_tokens[0]):
        sentence_type = 'headline-chapter'
        type_value = word_tokens[0] # chapter number
        forward_identified_sentence_type_list.append((str(line_index_number),sentence_type,type_value))

    else:
        sentence_type = 'none'
        type_value = 'none'
        forward_identified_sentence_type_list.append((str(line_index_number),sentence_type,type_value))

forward_identified_sentence_type_list

[('0', 'headline-chapter', '3'),
 ('1', 'none', 'none'),
 ('2', 'headline-section', '7'),
 ('3', 'part', '1'),
 ('4', 'part', '2'),
 ('5', 'part', '3'),
 ('6', 'none', 'none'),
 ('7', 'none', 'none'),
 ('8', 'none', 'none'),
 ('9', 'none', 'none'),
 ('10', 'none', 'none'),
 ('11', 'headline-section', '8'),
 ('12', 'part', '1'),
 ('13', 'part', '2'),
 ('14', 'part', '3'),
 ('15', 'none', 'none'),
 ('16', 'headline-section', '9'),
 ('17', 'part', '1'),
 ('18', 'sub-part', 'a'),
 ('19', 'none', 'none'),
 ('20', 'sub-part', 'b'),
 ('21', 'none', 'none'),
 ('22', 'sub-part', 'c'),
 ('23', 'none', 'none'),
 ('24', 'part', '2'),
 ('25', 'sub-part', 'a'),
 ('26', 'none', 'none'),
 ('27', 'sub-part', 'b'),
 ('28', 'none', 'none'),
 ('29', 'part', '3'),
 ('30', 'part', '4'),
 ('31', 'part', '5'),
 ('32', 'part', '6'),
 ('33', 'part', '7'),
 ('34', 'none', 'none'),
 ('35', 'headline-section', '10'),
 ('36', 'part', '1'),
 ('37', 'part', '2'),
 ('38', 'part', '3'),
 ('39', 'part', '4'),
 ('40', 'p

# Create XML content

In [103]:
from xml.etree import ElementTree, cElementTree
from xml.dom import minidom

root = ElementTree.Element('dokument')
#print(ElementTree.tostring(root))

element_text = ElementTree.SubElement(root, 'tekst')

In [104]:
element_regulation_title = ElementTree.SubElement(element_text, 'tittel')
element_regulation_title.text = "Regulation_Title"

In [105]:
element_kapittel = ElementTree.SubElement(element_text, 'kapittel', type="kapittel")

if "chapter_number" in forward_chapter_info:
    attribute_id = "/kapittel/" + forward_chapter_info["chapter_number"]
    element_kapittel.set('id', attribute_id)

#url = "https://lovdata.no/forskrift/2013-11-22-1404/k1"
#element_kapittel.set('url', url)

In [106]:
if "chapter_title" in forward_chapter_info:
    element_ktittel = ElementTree.SubElement(element_kapittel, "ktittel")
    element_ktittel.text = forward_chapter_info["chapter_title"]

In [109]:
previous_sentence_type_that_was_not_none = "none"

chapter_number = "undetected"
section_number = "undetected"
part_number = "undetected"
sub_part_id = "undetected"

element_section = ""
element_part = ""
element_sub_part = ""

for line_index_number, line_text in enumerate(forward_file_lines):

    # get metadata about sentence
    identified_sentence_type_as_tuple = forward_identified_sentence_type_list[line_index_number]
    identified_sentence_type = identified_sentence_type_as_tuple[1]
    identified_sentence_value = identified_sentence_type_as_tuple[2]

    ###

    if identified_sentence_type == "headline-chapter":
        chapter_number = identified_sentence_value

    elif identified_sentence_type == "headline-section":
        section_number = identified_sentence_value

        element_section = ElementTree.SubElement(element_kapittel, "paragraf")

        new_id = "/kapittel/" + chapter_number + "/paragraf/" + section_number
        element_section.set("id", new_id)

        element_ptittel = ElementTree.SubElement(element_section, "ptittel")
        element_ptittel.text = line_text.replace("\n","")

    elif identified_sentence_type == "part":
        part_number = identified_sentence_value

        element_part = ElementTree.SubElement(element_section, "ledd")
        element_part.text = line_text.replace("\n","")

        new_id = "/kapittel/" + chapter_number + "/paragraf/" + section_number + "/ledd/" + part_number
        element_part.set("id", new_id)

    elif identified_sentence_type == "sub-part":
        sub_part_id = identified_sentence_value

        element_sub_part = ElementTree.SubElement(element_part, "liste")

        new_id = "/kapittel/" + chapter_number + "/paragraf/" + section_number + "/ledd/" + part_number + "/liste/" + sub_part_id
        element_sub_part.set("id", new_id)

        element_sub_part_lverdi = ElementTree.SubElement(element_sub_part, "lverdi")
        element_sub_part_lverdi.text = identified_sentence_value

        element_sub_part_avsnitt = ElementTree.SubElement(element_sub_part, "avsnitt")
        element_sub_part_avsnitt.text = line_text.replace("\n","")

        

    





    ###

    # at the end of the loop
    if identified_sentence_type != "none":
        previous_sentence_type_that_was_not_none = identified_sentence_type
    else:
        if line_text != "\n":
            print("previous_sentence_type: " + previous_sentence_type_that_was_not_none)
            print(line_text) # TODO this must be added to the XML. Handle after everything else is completed.
    



previous_sentence_type: part
0	Amended by Regulation of 4 May 2015 No.. 445 (in force on 6 May 2015).

previous_sentence_type: headline-section
Non-combustible material means a material which, when tested and evaluated in accordance with MSC.61(67) (the FTP Code), satisfies the requirements for qualifying materials as non-combustible. The insulation shall, if necessary in order to preserve the fire-insulating effect, be mounted with a sheet of galvanized steel or aluminium.

previous_sentence_type: part
0	Amended by Regulation of 4 May 2015 No.. 445 (in force on 6 May 2015).

previous_sentence_type: headline-section
The vessel's machinery space shall have at least two ventilators capable of being closed from deck. Cabins shall have at least two ventilators with a diameter of at least 8 cm.

previous_sentence_type: headline-section
Materials, varnishes, paints, etc. produced on a nitrocellulose basis or any other basis which will readily ignite are not permitted to be used anywhere on b

# Create tree to XML file

In [108]:
tree = cElementTree.ElementTree(root) # wrap it in an ElementTree instance, and save as XML

# Since ElementTree write() has no pretty printing support, used minidom to beautify the xml
t = minidom.parseString(ElementTree.tostring(root)).toprettyxml()
tree1 = ElementTree.ElementTree(ElementTree.fromstring(t))

tree1.write("filename.xml",encoding='utf-8', xml_declaration=True)